In [39]:
import unittest
import numpy as np 
import importlib
import sys
sys.path.append('../')
from src import dissipation_tensor
importlib.reload(dissipation_tensor)
from src.dissipation_tensor import Dissipation_Tensor

In [40]:
class Dissipation_tests(unittest.TestCase):

    def test_dissipation_tensor_constructed_with_correct_size_and_initially_zeros(self):

        n = 9
        my_dissipation_tensor = Dissipation_Tensor(n, "general")
        d_tensor = my_dissipation_tensor.tensor
        first_layer_size = len(d_tensor)
        second_layer_size = len(d_tensor[0])
        inner_matrix_shape = np.shape(d_tensor[0][0].toarray())
        empty_array = np.zeros((n,n))
        self.assertEqual(first_layer_size, n)
        self.assertEqual(second_layer_size, n)
        self.assertEqual(inner_matrix_shape, (n,n))
        self.assertTrue(all(np.array_equal(d_tensor[i][j].toarray(), empty_array) for i in range(n) for j in range(n)))

    def test_creates_correct_tensor_select_type_of_Kossakowski_matrix_with_getter(self):
        
        n = 2 
        kossakowski_types = ["General", "Symmetric"] #can include diagonal but it's not a priority right now
        my_dissipation_tensor_general = Dissipation_Tensor(n, kossakowski_types[0])
        my_dissipation_tensor_symmetric = Dissipation_Tensor(n, kossakowski_types[1])

        self.assertEqual(kossakowski_types[0].lower(), my_dissipation_tensor_general.kossakowski_type)
        self.assertEqual(kossakowski_types[1].lower(), my_dissipation_tensor_symmetric.kossakowski_type)

    def test_validation_of_kossakowski_types(self):

        n = 2
        
        with self.assertRaises(Exception) as context:
            Dissipation_Tensor(n,"jhjhkjhkj")
        
        self.assertEqual(str(context.exception), "Kossakowski matrix type must be \"general\" or \"symmetric\"")




        


In [41]:
unittest.main(argv=[''], exit = False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
